In [15]:
from pipeline.training import SCORING, STRONG_CV
from pipeline.training import fit_pipeline, cross_validate_pipeline
from utils.feature_generation import get_data_with_feature
%config IPCompleter.use_jedi = False

In [16]:
import joblib

In [17]:
forest = joblib.load('./data/redir/best_rf.pkl')

/home/nizhikebinesi/online-store-redirects/venv/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/nizhikebinesi/online-store-redirects/venv/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [27]:
import pandas as pd
from pipeline.training import load_pipeline, score_pipeline
%config IPCompleter.use_jedi = False

pipe = load_pipeline("./models/tfidf_baseline1")
data = pd.read_csv("validation_data.csv")

In [18]:
import pandas as pd
validate = pd.read_csv("validation_data.csv")
validate = get_data_with_feature(validate, './data/redir')
validate['cosine_dist'] = validate['cosine_dist'].fillna(validate['cosine_dist'].mean())

X_validate = validate.drop(columns=['is_redirect'])
y_validate = validate['is_redirect']

In [167]:
import numpy as np
pred = (forest.predict_proba(X_validate)[:, 1] * 1/4 + pipe.predict_proba(data)[:, 1] *  3/4)
pred = (pred > 0.505).astype(np.int64)

In [168]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, confusion_matrix

print("\nMetrics for validate:")
for metric in [f1_score, accuracy_score]:
    print(metric.__name__, round(metric(y_validate, pred), 4))
    
print('Confusion matrix for validate:')
print(confusion_matrix(y_validate, pred))


Metrics for validate:
f1_score 0.6289
accuracy_score 0.6949
Confusion matrix for validate:
[[103  37]
 [ 35  61]]


In [172]:
pipe.predict_proba(data)[:, 1].argmax()

199

In [178]:
categories = pd.read_csv("./data/redir/categories.csv")
df = pd.read_csv("./data/redir/420_redirects.csv")

In [230]:
"""
Full Pipeline: A-B-C
Development Pipeline: B-C

A) Making negative examples

B) Offering categories for redirect(Filter)

C) Predict probability for (query, category) (redirects)
"""
import pandas as pd


class Preprocessor:
    pass


class Predictor:
    def __init__(self, model, categories):
        """

        :param model: sklearn-like model
        """
        self.model = model
        self.categories = categories

    def preprocess(self):
        # TODO
        pass

    def prepare_data(self, data):
        print(data)
        cats = pd.DataFrame()
        cats['id'] = data['categories_ids']

        df = pd.DataFrame()
        df['category_id'] = cats['id']
        df['category_name'] = cats.merge(self.categories)['name']
        df['query'] = [data['query']] * len(df)
        return df


    def predict_for_one_category(self, atom):
        """

        :param atom: [query: str, category_id: int]
        :return: float score
        """
        pass

    def predict_scores(self, data):
        """
        Метод для получения скоров для каждой категории
        (т.е. список скоров соответствует списку категорий)


        :param data: list/pd.DataFrame of query + category_id
        :return: list of float
        """
        pass

    def predict(self, data):
        """

        :param data:
        :return:

        Example of data:
        [{"query": "айфон X", "categories_ids": [1, 42, 142]}]

        Пример возвращаемого значения:
        [{"query": "айфон X", "category_id": 42, "category_name": "Техника Apple", "categories_preds": [0.99]}]
        """
        result = []
        for d in data:
            tmp = self.prepare_data(d)
            print(tmp)
            pred_old = self.model.predict_proba(tmp)[:, 1]
            pred = tmp.iloc[pred_old.argmax()]
            res = {
                'query': d['query'],
                'category_id': pred['category_id'],
                'category_name': pred['category_name'],
                'categories_preds': list(pred_old)
            }
            result.append(res)
        return result


In [231]:
p = Predictor(pipe, categories)

In [232]:
df.head(2)

,Unnamed: 0,redir_id,rule_id,start_date,query,category_id,parent_id,category
0,0,10303,23358,2020-08-19,dior хайлайтер,3327621,3327595.0,Хайлайтеры
1,1,10304,23359,2020-08-19,dior хайлатер,3327621,3327595.0,Хайлайтеры


In [233]:
# df['category_id'].unique()

In [234]:
d = [{"query": "dior хайлайтер", "categories_ids": [3327621, 3327622, 3327619, 3327959, 3328775, 3329045]}]

In [235]:
p.predict(d)

{'query': 'dior хайлайтер', 'categories_ids': [3327621, 3327622, 3327619, 3327959, 3328775, 3329045]}
   category_id    category_name           query
0      3327621       Хайлайтеры  dior хайлайтер
1      3327622  Туши для ресниц  dior хайлайтер
2      3327619             Губы  dior хайлайтер
3      3327959        Binturong  dior хайлайтер
4      3328775       BlendSMART  dior хайлайтер
5      3329045        BIO World  dior хайлайтер


[{'query': 'dior хайлайтер',
  'category_id': 3327622,
  'category_name': 'Туши для ресниц',
  'categories_preds': [0.9438322449500943,
   0.9859640181629619,
   0.9648794970354365,
   0.8428305945512996,
   0.8514966224563965,
   0.8847087510204606]}]